<a href="https://colab.research.google.com/github/helmana/DLeeg/blob/master/2ch_20s_Orthogonal1(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install mne

In [18]:

import matplotlib
from matplotlib import pyplot as plt
import numpy as np


import random
import mne
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
from mne import io
from mne import viz
#from mne.datasets import testing
from mne import Epochs, io, pick_types
from mne.event import define_target_events
from mne.time_frequency import psd_welch
print(__doc__)

Automatically created module for IPython interactive environment


In [19]:
subject_number=20


# load dataset in array
list_raw_fnames = [[0]*2]*subject_number
for x in range(subject_number):
    list_raw_fnames[x] = mne.datasets.eegbci.load_data(x+1,[1,2])

list_rawdata = np.zeros((subject_number,2), dtype='object')

for i in range(subject_number):
    for j in range(2):
        list_rawdata[i][j] = mne.io.read_raw_edf(list_raw_fnames[i][j], preload=True)
print( list_rawdata[:][:])

Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/physiobank/database/eegmmidb/S001/S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/physiobank/database/eegmmidb/S001/S001R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/physiobank/database/eegmmidb/S002/S002R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/physiobank/database/eegmmidb/S002/S002R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extr

In [20]:
import math


task_number=1
task_time = 60
sampel_number_per_sec =  160 # sampel rate
total_sampel_number =  sampel_number_per_sec *task_time # 60*160
sample_shift = 4 #step len
window_len= 20

# img for each one
subject_img_number = math.floor((total_sampel_number - sampel_number_per_sec) / sample_shift) +1 - window_len +1
test_start_index=subject_img_number - math.floor(subject_img_number/4) 

train_number =  test_start_index 
test_number = subject_img_number - test_start_index

print(train_number,test_number)


1757 585


In [21]:
ch_number = 2

train_img = np.zeros((train_number*subject_number, ch_number, window_len + 1, 160), dtype = float)
train_label =[]


test_img = np.zeros((test_number*subject_number, ch_number, window_len, 160), dtype = float)
test_label =[]


print(train_img.shape)
print(test_img.shape)

(35140, 2, 21, 160)
(11700, 2, 20, 160)


In [0]:
def normalize_channel_data(ch , i, ch_min, ch_max):
    ch = ((ch - ch_min[i]) / (ch_max[i] - ch_min[i] ) )
    return ch

def ProjectionVector(VecA,VecB):
    Projeted_VecB_on_VecA=np.dot(VecA,VecB)/np.dot(VecA,VecA)*VecA
    return Projeted_VecB_on_VecA

In [0]:
list_rawdataTest=list_rawdata.copy()
for s in range(subject_number):
    
    DataChannelsRaw, times =list_rawdataTest[s][0][:64,:9600]
    
    # make a copy
    DataChannelsRaw_t=DataChannelsRaw.copy()
    times_t=times.copy()
    
    # find min/max per channel
    ch_max =[]
    ch_min =[]
    for p in range(len(DataChannelsRaw_t)):
        ch_max = np.append(ch_max, max(DataChannelsRaw_t[p])) # max for each cannel
        ch_min = np.append(ch_min, min(DataChannelsRaw_t[p])) # min for each cannel
        
    # Normalize Channels
    DataChannelsNormal=np.zeros((ch_number,times.size),dtype = float)
    for i in range(ch_number):
        DataChannelsNormal[i]=normalize_channel_data(DataChannelsRaw_t[i], i, ch_min, ch_max)
    
    # Orthogonal Second Channel
    DataChannelsOrt=np.zeros((ch_number,times.size),dtype = float)
    DataChannelsOrt[0]=DataChannelsNormal[0]
    DataChannelsOrt[1]=DataChannelsNormal[1]-ProjectionVector(DataChannelsNormal[0],DataChannelsNormal[1])
    
    
    for j in range (subject_img_number):
        for i in range(ch_number):
            for z in range (window_len):
                ExtractedData=DataChannelsOrt[i,0+(j+z)*sample_shift:sampel_number_per_sec+(j+z)*sample_shift]

                if j <test_start_index :
                    train_img[s*train_number + j][i][z] = ExtractedData
                    if z==window_len-1 :
                      train_img[s*train_number + j][i][z+1] = s
                      
                else:
                    test_img[s*test_number + j - test_start_index][i][z] = ExtractedData
        if j >=test_start_index :
          test_label = np.append(test_label, (s))

           

In [24]:
#shaffle train img array
train_img_shuffle = train_img.copy()

np.random.shuffle(train_img_shuffle)

train_img_2 = np.zeros((train_number*subject_number, ch_number, window_len, 160), dtype = float)
train_img_2.shape

(35140, 2, 20, 160)

In [0]:
# img label 

train_img_shuffle_len =len(train_img_shuffle)
for i in range(train_img_shuffle_len):
    train_label = np.append(train_label, (train_img_shuffle[i][0][window_len][0] ))
    for j in range (ch_number):
      train_img_2[i][j] = np.delete(train_img_shuffle[i][j], window_len, axis=0)

In [0]:
import keras
from keras.utils import to_categorical

In [0]:
train_label = to_categorical(train_label, subject_number)
test_label = to_categorical(test_label, subject_number)

In [28]:
# valid & train
x_train =train_img_2[:20000]
y_train =train_label[:20000]

x_valid =train_img_2[20000:]
y_valid =train_label[20000:]

x_train.shape

(20000, 2, 20, 160)

In [29]:
from keras import layers
from keras import models
from keras import regularizers


model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation = 'relu', input_shape = (ch_number,window_len,160), data_format= "channels_first" ))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))
print(model.output.shape)
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
print(model.output.shape)
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.5))
print(model.output.shape)
model.add(layers.Dense(subject_number, activation = 'softmax'))
print(model.output.shape)

(?, 64, 18, 158)
(?, 32, 9, 158)
(?, 30, 7, 128)
(?, ?)
(?, 512)
(?, 20)


In [0]:

from keras import optimizers

model.compile(loss= 'categorical_crossentropy',
              optimizer= optimizers.RMSprop(lr= 1e-4),
              metrics = ['acc'])

In [31]:
history = model.fit(
        x_train,
        y_train,
        epochs = 15,
        batch_size = 10,
        validation_data = (x_valid, y_valid)
)

Train on 20000 samples, validate on 15140 samples
Epoch 1/15
20000/20000 [==============================] - 17s 841us/step - loss: 2.0677 - acc: 0.3636 - val_loss: 0.9797 - val_acc: 0.7437
Epoch 2/15
20000/20000 [==============================] - 16s 814us/step - loss: 0.8090 - acc: 0.7352 - val_loss: 0.3737 - val_acc: 0.9043
Epoch 3/15
20000/20000 [==============================] - 16s 817us/step - loss: 0.3882 - acc: 0.8738 - val_loss: 0.3694 - val_acc: 0.8701
Epoch 4/15
20000/20000 [==============================] - 16s 816us/step - loss: 0.2414 - acc: 0.9214 - val_loss: 0.4821 - val_acc: 0.8144
Epoch 5/15
20000/20000 [==============================] - 16s 821us/step - loss: 0.1765 - acc: 0.9417 - val_loss: 0.1020 - val_acc: 0.9735
Epoch 6/15
20000/20000 [==============================] - 16s 819us/step - loss: 0.1343 - acc: 0.9537 - val_loss: 0.0680 - val_acc: 0.9804
Epoch 7/15
20000/20000 [==============================] - 16s 816us/step - loss: 0.1091 - acc: 0.9643 - val_loss: 0.

In [32]:
model.evaluate(test_img, test_label)


11700/11700 [==============================] - 1s 106us/step


[0.16813713815294412, 0.9634188034188034]